In [ ]:
#Import a few libraries
import glob
from pathlib import Path
import os
import sys
import numpy as np
from scipy.io import loadmat
import ray
from MAPIT.core import StatsProcessor

In [ ]:
F='path/to/csv/folder'

---

<div align="center">

#  Data import 
### The following blocks of code will change based on individual use cases

</div>



---

In [ ]:
#This sets a path object for the 
#inputs, inventories, and outputs
indir = os.path.join(F,'input')
invdir = os.path.join(F,'inventory')
outdir = os.path.join(F,'output')


In [ ]:
#Use glob to get list of files
#For each measurement type
infiles = glob.glob(indir+'/*.csv')
invfiles = glob.glob(invdir+'/*.csv')
outfiles = glob.glob(outdir+'/*.csv')

In [ ]:
#This just makes sure that the matrix shape
#of the csv data is 
# samples, features (time, values for MAPIT)
def checkImportAxes(d):
  if d.shape[0] < d.shape[1]:
    np.swapaxes(d,0,1)
  
  return d

In [ ]:
#Read in the csv data
#And store it in a list (required)
indat = []
invdat = []
outdat = []

inTdat = []
invTdat = []
outTdat = []

for i in range(len(infiles)):
    z = checkImportAxes(np.loadtxt(infiles[i],delimiter=','))
    inTdat.append(z[:,0])
    indat.append(z[:,1])

for i in range(len(invfiles)):
    z = checkImportAxes(np.loadtxt(invfiles[i],delimiter=','))
    invTdat.append(z[:,0])
    invdat.append(z[:,1])

for i in range(len(outfiles)):
    z = checkImportAxes(np.loadtxt(outfiles[i],delimiter=','))
    outTdat.append(z[:,0])
    outdat.append(z[:,1])

In [ ]:
#alternative loading showing how you might load .mat data
#largely based on SSPM-based measurement blocks
#note the squeeze_me
#TODO: upload data from desktop
# F = os.path.join(x, 'debug_data','csv')
# x1 = loadmat(F,squeeze_me=True)

# invdat = x1['invn']['data']
# inpdatt = x1['in']['data']
# outdat = x1['outn']['data']

# invTdat = x1['invn']['time']
# inTdat = x1['in']['time']
# outTdat = x1['outn']['time']

#### Note that all of the data and time variables are lists that contain arrays. This is because MAPIT supports variable sampling so there might be different numbers of samples for different locations. However, the data, time array pairs should be the same length at each location. For example ``indat[0].shape = inTdat[0].shape``

In [ ]:
#Note here that all the data and time
#variables are lists that contain arrays,
#because MAPIT supports variable sampling,  
print(type(indat))
print(type(invdat))
print(type(outdat))

print(type(inTdat))
print(type(invTdat))
print(type(outTdat))

The next block specifies relative standard deviation which is stored in `ErrorMatrix` with `(nlocs,2)`. `nlocs` reflects the total number of locations at each location. The first value in the second dimension (e.g., `ErrorMatrix[:,0]`) refers to the random error whereas the second entry (e.g., `ErrorMatrix[:,1]`) refers to the systematic error.

Important: ErrorMatrix expects relative standard deviation (RSD) in decimal form, *not* percentage. For example, a 10% relative standard deviation should be specified as 0.10 

Note that for this example, a uniform RSD is specified at 0.5% for all locations

A `TotalErrorMatrix` is also specified, which is required for certain statistical operations. It is expected that the locations will be stacked as (inputs, inventories, outputs)


In [ ]:
inputErrorMatrix = np.ones((len(indat),2)) * 0.005 
inventoryErrorMatrix = np.ones((len(invdat),2)) * 0.005 
outputErrorMatrix = np.ones((len(outdat),2)) * 0.005 
IT = 25
mbaTime = 416

In [ ]:
MB1 = StatsProcessor.MBArea(rawInput = indat,
                                rawInventory = invdat,
                                rawOutput = outdat,
                                rawInputTimes = inTdat,
                                rawInventoryTimes = invTdat,
                                rawOutputTimes = outTdat,
                                inputErrorMatrix = inputErrorMatrix,
                                inventoryErrorMatrix = inventoryErrorMatrix,
                                outputErrorMatrix = outputErrorMatrix,
                                mbaTime = mbaTime, 
                                iterations = 400,
                                ncpu=4,
                                nbatch=100,
                                dopar=True,
                                doTQDM=True) 

In [ ]:
MUF = MB1.calcMUF() 

In [ ]:
CUMUF = MB1.calcCUMUF()

In [ ]:
AI = MB1.calcActiveInventory() #5.3

In [ ]:
SEMUF, _, _, _ = MB1.calcSEMUF() 

In [ ]:
SEMUFAI, _, _ = MB1.calcSEMUFAI()

In [ ]:
MB1.nbatch=100
SITMUF = MB1.calcSITMUF() 

In [ ]:
Page = MB1.calcPageTT()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(Page.T)

In [ ]:
ray.shutdown()